# 2장. 데이터 병합 & 조인

In [1]:
import pandas as pd
import numpy as np

---
---

## 01. pd.merge()
- SQL의 join처럼 특정한 column을 기준으로 병합
- join 방식 : how 파라미터를 통해 명시
    - inner: default, 일치하는 값이 있는 경우. 즉, 양쪽 데이터가 다 존재해야 함
    - left : 왼쪽 기준
    - right : 오른쪽 기준
    - outer : left + right
    - 인덱스로 조인하기

In [9]:
customer = pd.DataFrame({'customer_id':np.arange(6),
                         'name':['jack','Tom','Merry','Leo','Cris','Hwang'],
                         'age':[40,30,20,10,15,25]})
customer

,customer_id,name,age
0,0,jack,40
1,1,Tom,30
2,2,Merry,20
3,3,Leo,10
4,4,Cris,15
5,5,Hwang,25


In [10]:
orders = pd.DataFrame({'customer_id' : [1, 1, 2, 2, 2, 3, 3, 1, 4, 9], 
                    'item' : ['치약', '칫솔', '이어폰', '헤드셋', '수건', '생수', '수건', '치약', '생수', '케이스'], 
                    'quantity' : [1, 2, 1, 1, 3, 2, 2, 3, 2, 1]})
orders

,customer_id,item,quantity
0,1,치약,1
1,1,칫솔,2
2,2,이어폰,1
3,2,헤드셋,1
4,2,수건,3
5,3,생수,2
6,3,수건,2
7,1,치약,3
8,4,생수,2
9,9,케이스,1


- inner join
    - default 방식으로 겹치지 않는 데이터 제외

In [11]:
pd.merge(customer, orders, on='customer_id')

,customer_id,name,age,item,quantity
0,1,Tom,30,치약,1
1,1,Tom,30,칫솔,2
2,1,Tom,30,치약,3
3,2,Merry,20,이어폰,1
4,2,Merry,20,헤드셋,1
5,2,Merry,20,수건,3
6,3,Leo,10,생수,2
7,3,Leo,10,수건,2
8,4,Cris,15,생수,2


- left join
    - 왼쪽 기준으로 결합

In [13]:
pd.merge(customer, orders, on='customer_id', how='left')

,customer_id,name,age,item,quantity
0,0,jack,40,NaN,NaN
1,1,Tom,30,치약,1.0
2,1,Tom,30,칫솔,2.0
3,1,Tom,30,치약,3.0
4,2,Merry,20,이어폰,1.0
5,2,Merry,20,헤드셋,1.0
6,2,Merry,20,수건,3.0
7,3,Leo,10,생수,2.0
8,3,Leo,10,수건,2.0
9,4,Cris,15,생수,2.0


- right join
    - 오른쪽 기준으로 병합

In [14]:
pd.merge(customer, orders, on="customer_id", how='right')

,customer_id,name,age,item,quantity
0,1,Tom,30.0,치약,1
1,1,Tom,30.0,칫솔,2
2,1,Tom,30.0,치약,3
3,2,Merry,20.0,이어폰,1
4,2,Merry,20.0,헤드셋,1
5,2,Merry,20.0,수건,3
6,3,Leo,10.0,생수,2
7,3,Leo,10.0,수건,2
8,4,Cris,15.0,생수,2
9,9,NaN,NaN,케이스,1


- outer join
    - 오른쪽 + 왼쪽 = 전체

In [16]:
pd.merge(customer, orders, on='customer_id', how='outer')

,customer_id,name,age,item,quantity
0,0,jack,40.0,NaN,NaN
1,1,Tom,30.0,치약,1.0
2,1,Tom,30.0,칫솔,2.0
3,1,Tom,30.0,치약,3.0
4,2,Merry,20.0,이어폰,1.0
5,2,Merry,20.0,헤드셋,1.0
6,2,Merry,20.0,수건,3.0
7,3,Leo,10.0,생수,2.0
8,3,Leo,10.0,수건,2.0
9,4,Cris,15.0,생수,2.0


- index 기준으로 join 하기

In [20]:
# 각 데이터프레임에 customer_id 컬럼을 인덱스로 지정해줌
cust1 = customer.set_index('customer_id')
order1 = orders.set_index('customer_id')

In [25]:
# 아래 두가지 방법으로 join 가능
pd.merge(cust1,order1, left_index=True, right_index=True)
pd.merge(cust1,order1, left_index=True, on = 'customer_id')

,name,age,item,quantity
customer_id,,,,
1,Tom,30,치약,1
1,Tom,30,칫솔,2
1,Tom,30,치약,3
2,Merry,20,이어폰,1
2,Merry,20,헤드셋,1
2,Merry,20,수건,3
3,Leo,10,생수,2
3,Leo,10,수건,2
4,Cris,15,생수,2


---
---

## 02. pd.concat()
- DataFrame을 병합시킬 때 쓰이는 함수
- axis
    - axis=1 => 열 단위 병합
    - axis=0 => 행 단위 병합

In [29]:
df1 = pd.DataFrame({'key1':np.arange(10),'value1':np.random.randn(10)})
df2 = pd.DataFrame({'key1':np.arange(10),'value1':np.random.randn(10)})
df1

,key1,value1
0,0,-0.792200
1,1,-0.550297
2,2,0.297346
3,3,-0.167697
4,4,2.107970
5,5,-0.854902
6,6,-0.421702
7,7,-1.323230
8,8,0.490902
9,9,1.232715


- 열 단위 결합(axis=0)(default값)
    - column 명이 다른 경우에는 컬럼이 추가되어 위아래에 NaN값이 채워진다.

In [30]:
# 그냥 기본 결합
pd.concat([df1,df2])

,key1,value1
0,0,-0.792200
1,1,-0.550297
2,2,0.297346
3,3,-0.167697
4,4,2.107970
5,5,-0.854902
6,6,-0.421702
7,7,-1.323230
8,8,0.490902
9,9,1.232715


In [31]:
# ignore_index = True를 사용하여 새로운 인덱스 호출
pd.concat([df1,df2], ignore_index=True)

,key1,value1
0,0,-0.792200
1,1,-0.550297
2,2,0.297346
3,3,-0.167697
4,4,2.107970
5,5,-0.854902
6,6,-0.421702
7,7,-1.323230
8,8,0.490902
9,9,1.232715


- 행 단위 결합(axis=1)
    - 컬럼명이 다르더라도 그냥 새로운 컬럼이 생기는 것이기 때문에 컬럼명이 같은 경우와 차이가 없다.

In [34]:
pd.concat([df1,df2],axis=1)

,key1,value1,key1,value1
0,0,-0.792200,0,0.693655
1,1,-0.550297,1,0.742771
2,2,0.297346,2,2.131462
3,3,-0.167697,3,-0.962528
4,4,2.107970,4,-1.380083
5,5,-0.854902,5,-1.138494
6,6,-0.421702,6,-1.425359
7,7,-1.323230,7,1.080685
8,8,0.490902,8,0.236680
9,9,1.232715,9,1.428652
